# Calculating Betas for the Market's Stocks #

### Calculating the Betas for all the Stocks in the Universe ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Statistics
from scipy.stats import norm

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from capm_toolkit import capm_regression
from capm_toolkit import rolling_capm_regression
from capm_toolkit import compute_factor_contributions
from capm_toolkit import compute_residual_returns

In [2]:
# Get the important data for the Risk-Free Rate
rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index)
rfr.dropna(inplace = True)

# Get the important data for the S&P500
data_sp500 = pd.read_csv(r'..\additional_data\sp500.csv')
data_sp500.set_index('Date', inplace=True)
data_sp500.index = pd.to_datetime(data_sp500.index)
data_sp500 = data_sp500['sp_500']

In [3]:
# Import Universe
# tickers = pd.read_csv(r"..\additional_data\tickers.csv")
# tickers = tickers['tickers']

# Download all the data
# DataFrame to store everything
# df_returns = pd.DataFrame()

# for ticker in tickers:
#    try:
#        df = import_daily_financial_data(
#            ticker=ticker, 
#            start_date='2015-01-01', 
#            end_date='2025-01-01', 
#            returns=True
#        )
#        
#        returns = df['returns'].rename(ticker)
#        
#        df_returns = pd.concat([df_returns, returns], axis=1)
#        
#        print(f'Data Ready for {ticker}')
#        
#    except Exception as e:
#         print(f'Error reading {ticker}: {e}')

In [4]:
# Import Data
df_returns = pd.read_csv(r'..\additional_data\stocks_returns.csv')
df_returns = df_returns.rename(columns={'Unnamed: 0':'Date'})
df_returns.set_index('Date', inplace=True)
df_returns.index = pd.to_datetime(df_returns.index)

df_returns

,A,AAPL,ABBV,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WST,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.018915,-0.028576,-0.018998,0.000222,-0.004989,-0.018352,-0.035215,-0.000721,-0.014722,-0.010052,...,-0.013900,0.000000,-0.017166,-0.011416,-0.027743,-0.064259,-0.020527,0.036625,-0.014177,-0.006022
2015-01-06,-0.015700,0.000094,-0.004962,-0.011421,-0.020350,-0.023753,-0.019911,-0.008324,-0.019973,-0.011709,...,-0.017255,-0.001108,-0.021562,0.005306,-0.005330,-0.005898,-0.012351,-0.008513,-0.007231,-0.009804
2015-01-07,0.013184,0.013925,0.039621,0.008076,0.008190,0.010462,0.014920,0.008685,-0.002089,0.007747,...,0.016088,0.003045,0.028945,0.008598,0.010081,0.007857,0.032601,0.024566,0.025146,0.020432
2015-01-08,0.029534,0.037702,0.010404,0.020345,0.025135,0.017501,-0.014717,0.022681,0.024446,0.003522,...,0.019467,0.010722,0.011959,0.011532,0.016508,0.007517,0.017270,0.010561,0.021134,0.015282
2015-01-09,-0.007364,0.001072,-0.027736,-0.010563,-0.014922,-0.000731,-0.008566,-0.007188,-0.009054,-0.011270,...,-0.011519,-0.001368,0.012422,-0.012361,-0.001411,-0.019043,-0.001768,-0.014529,0.014508,0.001583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,0.011083,0.011413,0.008985,0.003929,0.002683,0.014782,0.004355,0.007822,0.012494,0.013291,...,0.009663,0.006390,0.007575,0.007210,0.000940,0.005432,0.008480,0.004484,0.012187,0.002553
2024-12-26,-0.001986,0.003171,-0.004454,0.004434,0.004944,-0.000138,-0.000395,0.002594,-0.003159,-0.002203,...,0.001981,-0.003190,0.005952,-0.000440,0.000845,0.002705,0.007380,-0.000466,0.003559,0.004966
2024-12-27,-0.002137,-0.013331,-0.006663,-0.002432,-0.008208,-0.004919,-0.000593,-0.003539,-0.008999,-0.006305,...,-0.000630,-0.006410,-0.007079,-0.000587,-0.000094,-0.007966,-0.005678,-0.003549,-0.019799,-0.005574


In [14]:
tickers = df_returns.columns

In [17]:
# Let us get the betas of each stock

alphas_dict = {}
betas_dict = {}
sigma_dict = {}

# Create the Loop to Obtain the Betas
for ticker in tickers:
    df = rolling_capm_regression(
        df_returns[ticker], 
        sp500, 
        rfr['risk_free_rate']
    )
    alphas_dict[ticker] = df['alpha']
    betas_dict[ticker] = df['beta']
    sigma_dict[ticker] = df['sigma']

    print(f'{ticker} is ready.')

In [18]:
# Create Alpha's DataFrame
alpha_df = pd.DataFrame.from_dict(alphas_dict)
# alpha_df = alpha_df.apply(lambda x: x.fillna(x.mean()), axis=0)

alpha_df

In [19]:
# Create Beta's DataFrame
betas_df = pd.DataFrame.from_dict(betas_dict)
# betas_df = betas_df.apply(lambda x: x.fillna(x.mean()), axis=0)

betas_df

In [20]:
# Create the Sigma's DataFrame
sigma_df = pd.DataFrame.from_dict(sigma_dict)
# sigma_df = sigma_df.apply(lambda x: x.fillna(x.mean()), axis=0)

sigma_df

In [21]:
# Save the betas

alpha_df.to_csv(r"..\additional_data\capm_halpha.csv")
betas_df.to_csv(r"..\additional_data\capm_hbetas.csv")
sigma_df.to_csv(r"..\additional_data\capm_hsigma.csv")

In [28]:
# Calculate the Market Excess Returns
market_excess_returns = sp500 - rfr['risk_free_rate']

market_excess_returns

In [29]:
# Calculate Stocks Excess Returns
df_excess_returns = df_returns.sub(rfr['risk_free_rate'], axis=0)

df_excess_returns

In [30]:
# Common Indexes
common_index = df_excess_returns.index.intersection(market_excess_returns.index)

# Reindex
df_excess_returns, market_excess_returns = (
    df_excess_returns.loc[common_index], 
    market_excess_returns.loc[common_index]
)

print(df_excess_returns.shape)
print(market_excess_returns.shape)

In [31]:
# Set the Window
window = len(df_excess_returns)

# Create the Betas and Alpha + Residuals DataFrames for the whole time stamp
capm_betas_dict = {}

# Loop to Obtain Betas and Alpha + Residuals
for ticker in df_excess_returns.columns:
    
    # Fit the WLS model
    model = capm_regression(
        df_excess_returns[ticker], 
        market_excess_returns,
        window=window,
        WLS=True
    )

    # Extract Alpha and Beta
    alpha = model.params.iloc[0]
    beta = model.params.iloc[1]

    # Store Beta
    betas_dict[ticker] = beta

# Create Beta Series
betas_series = pd.Series(betas_dict)

betas_series

In [32]:
# Plot
ticker = 'WMT'

# Mean
mean = betas_df[ticker].mean()

# Create the Plot
plt.figure(figsize=(10, 6))
plt.plot(betas_df[ticker], label=f'{ticker} Beta', color='blue', alpha=0.7)
plt.axhline(y=betas_series.loc[ticker], color='red', linestyle='dashed', label=f'{ticker} Historical Beta')
plt.axhline(y=mean, color='black', linestyle='dashed', label=f'{ticker} Mean Beta')

# Config
plt.title('Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show

plt.show()

In [33]:
# Compare Volatility and Betas for each stock

annualized_vol = df_returns[ticker].std() * np.sqrt(252)
comparison = betas_series.loc[ticker]/annualized_vol

print(f'{ticker} Historical Beta is: {betas_series.loc[ticker]}')
print(f'{ticker} Historical Annualized Standard Deviation is: {annualized_vol}')
print(f'{ticker} Ratio Between Both is: {comparison}')

In [34]:
# Calculate Mean and Standard Deviation
mu = betas_series.mean()
sigma = betas_series.std()

# Create Histogram
plt.figure(figsize=(10, 6))
plt.hist(betas_series, bins=30, density=True, color='lightskyblue', alpha=0.5, edgecolor='black', label='Betas Distribution')

# Generate the Values of the Normal Distribution
x = np.linspace(betas_series.min(), betas_series.max(), 100)
y = norm.pdf(x, mu, sigma)

# Graph the Real Normal Distribution
plt.plot(x, y, color='black', linestyle='solid', linewidth=2, label='Normal Distribution')

# Reference Lines
plt.axvline(x=mu, color='black', linestyle='dashed', label='Mean Returns')
plt.axvline(x=betas_series.median(), color='red', linestyle='dashed', label='Median Returns')
plt.axvline(x=mu + sigma, color='grey', linestyle='dashed')
plt.axvline(x=mu - sigma, color='grey', linestyle='dashed')

# Config
plt.title('Betas Histogram with Normal Distribution')
plt.xlabel('Return')
plt.ylabel('Density')

# Legends and Grid
plt.legend()
plt.grid(True)

# Show
plt.show()

### Comparing Residuals ###

In [42]:
# Compute Residuals in the Alternative Way

stock = 'WMT'

# Common Index
common_index = df_excess_returns.index.intersection(betas_df.index).intersection(alpha_df.index).intersection(market_excess_returns.index)

# Cut DataFrames
r_i = df_excess_returns[stock].loc[common_index]
beta = betas_df[stock].loc[common_index]
r_m = market_excess_returns.loc[common_index]

# Calculate factor returns
r_f = compute_factor_contributions(r_m, beta)

# Calculate residual returns
residual_returns = compute_residual_returns(r_i, r_m, beta)
residual_returns.name = 'residual_returns'

residual_returns

In [43]:
# Create the Plot
plt.figure(figsize=(10, 6))
plt.plot(r_i.cumsum(), label=f'{stock} Returns', alpha=0.7)
plt.plot(r_f.cumsum(), label=f'{stock} Factor Returns', alpha=0.7)
plt.plot(residual_returns.cumsum(), label=f'{stock} Residual Returns', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()

# Show
plt.show()

The method using the rolling betas adds so much noise to the calculation of the residual returns, especially because sometimes the alpha coefficient has a bias. So we prefer to use the residuals of the whole time stamp regression.

In [37]:
print(df_excess_returns.shape)
print(market_excess_returns.shape)
print(betas_df.shape)

In [38]:
# Calculate the Residual Returns for every stock
residual_returns_dict = {}

# Create the Loop to Obtain the Betas
for ticker in tickers:
    df = compute_residual_returns(
        df_excess_returns[ticker].loc[common_index], 
        market_excess_returns.loc[common_index],
        betas_df[ticker]
    )
    
    residual_returns_dict[ticker] = df

# Create the DataFrame
df_residual_returns = pd.DataFrame.from_dict(residual_returns_dict)

df_residual_returns['NVDA']

In [39]:
# Save the data
betas_series.to_csv(r"..\additional_data\capm_sbetas.csv")
df_residual_returns.to_csv(r"..\additional_data\capm_residual_returns.csv")